In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/data.csv').dropna()

print(df.shape)
print(df.columns)

(25697, 25)
Index([u'action_type', u'combined_shot_type', u'game_event_id', u'game_id',
       u'lat', u'loc_x', u'loc_y', u'lon', u'minutes_remaining', u'period',
       u'playoffs', u'season', u'seconds_remaining', u'shot_distance',
       u'shot_made_flag', u'shot_type', u'shot_zone_area', u'shot_zone_basic',
       u'shot_zone_range', u'team_id', u'team_name', u'game_date', u'matchup',
       u'opponent', u'shot_id'],
      dtype='object')


In [25]:
df.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5
5,Jump Shot,Jump Shot,244,20000012,34.0553,-145,-11,-118.4148,9,3,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,6


## Information about the data
As we can see, there are 25,697 labelled shots and 25 columns (one column is what we're trying to predict, which is whether or not he made the shot). Here are brief descriptions of what each column means:

- action_type:

Values: Jump Shot, Driving Dunk, Layup Shot, Running Jump Shot, etc.

This basically means the type of shot taken


- combined_shot_type

Values: Jump Shot, Dunk, Layup

Same thing as action_type but more generalized. action_type will say "Running Jump Shot" while combined_shot_type will say "Jump Shot," so action_type is more descriptive.

- game_event_id, game_id, shot_id

Identifiers

- loc_x, loc_y

(x,y) location of the position where the shot was taken. (0,0) refers to the location of the basket

- lat, lon

GPS coordinates of the stadium where the game was played

- minutes_remaining

Minutes remaining in the quarter

- period

The quarter

- playoffs

Flag for whether or not this was a playoff game

- season

The year of the game

- seconds_remaining

Seconds remaining in the minute

- shot_distance

Euclidean distance from the basket in feet

- shot_type

2 PT or 3 PT shot (this can also be determined by shot distance)

- shot_zone_area

Side of the court

- shot_zone_basic

Descriptive locations of where the shots are taken (Restricted, In the paint, Mid-Range, 

- shot_zone_range

Range of where the shot was taken

- team_id, team_name

ID, name of the team (Kobe's team)

- game_date

Date of the game

- matchup

The away vs home team

- opponent

The name of the opponent team

- shot_made_flag

Flag for if the shot is made or not

As you can see, dropping these redundant 

In [45]:
from sklearn import preprocessing

drop_list = ['combined_shot_type', 'game_event_id', 'game_id', 'lat', 'lon', 'shot_type', 'shot_zone_area',
            'shot_zone_basic', 'shot_zone_range', 'team_id', 'team_name', 'shot_id', 'opponent', 'matchup']

def home_game(matchup):
    if '@' in matchup:
        return 0
    else:
        return 1

def process_data(df):
    # Convert the action types
    le_action = preprocessing.LabelEncoder()
    le_opp = preprocessing.LabelEncoder()
    df['action_type'] = le_action.fit_transform(df['action_type'].values)
    df['opponent'] = le_opp.fit_transform(df['opponent'].values)
    df['home'] = df['matchup'].apply(home_game)
    df = df.drop(drop_list, axis=1) # Drop unwanted columns

    return df
    
df_p = process_data(df)

y = df['shot_made_flag']
X = df.drop('shot_made_flag', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

df_p.head(100)

,action_type,loc_x,loc_y,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,game_date,home
1,25,-157,0,10,1,0,2000-01,22,15,0.0,2000-10-31,0
2,25,-101,135,7,1,0,2000-01,45,16,1.0,2000-10-31,0
3,25,138,175,6,1,0,2000-01,52,22,0.0,2000-10-31,0
4,4,0,0,6,2,0,2000-01,19,0,1.0,2000-10-31,0
5,25,-145,-11,9,3,0,2000-01,32,14,0.0,2000-10-31,0
6,26,0,0,8,3,0,2000-01,52,0,1.0,2000-10-31,0
8,25,-65,108,6,3,0,2000-01,12,12,1.0,2000-10-31,0
9,40,-33,125,3,3,0,2000-01,36,12,0.0,2000-10-31,0
10,25,-94,238,1,3,0,2000-01,56,25,0.0,2000-10-31,0
11,25,121,127,11,1,0,2000-01,0,17,1.0,2000-11-01,1


In [5]:
import xgboost

In [6]:
results = {}
for n_estimators in range(10, 100, 10):
    rfcls = RandomForestClassifier(n_estimators=n_estimators)
    rfcls.fit(X_train, y_train)
    test_score = rfcls.score(X_test, y_test)
    train_score = rfcls.score(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [15]:
list(range(10, 100, 10))

[10, 20, 30, 40, 50, 60, 70, 80, 90]

0.97513940520446096

In [13]:
for dtcls in rfcls.estimators_:
    print(dtcls.tree_)